In [1]:
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Flatten, Dense,BatchNormalization,Activation,Dropout,LeakyReLU
from tensorflow.keras.optimizers import SGD,Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical
from tensorflow.keras.datasets import cifar100



(x_train, Y_train), (x_test, Y_test) = cifar100.load_data()
#x_train = x_train.astype('float32') / 255
#x_test = x_test.astype('float32') / 255

y_train = to_categorical(Y_train)
y_test = to_categorical(Y_test)

aug_data=ImageDataGenerator(rotation_range=20,horizontal_flip=True,width_shift_range=0.1,shear_range = 0.2,height_shift_range=0.1,zoom_range=0.2,brightness_range = (0.5, 1.5))
aug_data.fit(x_train)

169009152/169001437 [==============================] - 6s 0us/step


In [2]:

#import tensorflow as 
model = Sequential()

# Creating first block- (2 Convolution + 1 Max pool)
model.add(Conv2D(filters= 64, kernel_size= (3,3), strides= (1,1), padding='same', input_shape= (32, 32, 3)))
model.add(LeakyReLU(alpha=0.1))
model.add(Conv2D(filters= 64, kernel_size= (3,3), strides= (1,1), padding='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPool2D(pool_size= (2,2), strides=(1,1)))

# Creating second block- (2 Convolution + 1 Max pool)
model.add(Conv2D(filters= 128, kernel_size= (3,3), strides= (1,1), padding='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(Conv2D(filters= 128, kernel_size= (3,3), strides= (1,1), padding='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPool2D(pool_size= (2,2), strides=(1,1)))


# Creating third block- (3 Convolution + 1 Max pool)
model.add(Conv2D(filters= 256, kernel_size= (3,3), strides= (1,1), padding='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(Conv2D(filters= 256, kernel_size= (3,3), strides= (1,1), padding='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(Conv2D(filters= 256, kernel_size= (3,3), strides= (1,1), padding='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPool2D(pool_size= (2,2), strides=(1,1)))

# Creating fourth block- (3 Convolution + 1 Max pool)
model.add(Conv2D(filters= 512, kernel_size= (3,3), strides= (1,1), padding='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(Conv2D(filters= 512, kernel_size= (3,3), strides= (1,1), padding='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(Conv2D(filters= 512, kernel_size= (3,3), strides= (1,1), padding='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPool2D(pool_size= (2,2), strides=(2,2)))

# Creating fifth block- (3 Convolution + 1 Max pool)
model.add(Conv2D(filters= 512, kernel_size= (3,3), strides= (1,1), padding='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(Conv2D(filters= 512, kernel_size= (3,3), strides= (1,1), padding='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(Conv2D(filters= 512, kernel_size= (3,3), strides= (1,1), padding='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPool2D(pool_size= (2,2), strides=(1,1)))

# Flattening the pooled image pixels
model.add(Flatten())

# Creating 2 Dense Layers
model.add(Dense(units= 512))
model.add(LeakyReLU(alpha=0.1))

model.add(Dense(units= 100, activation='softmax'))


#adam=Adam(learning_rate=0.01,clipnorm=1,name='adam')
sgd=SGD(learning_rate=0.01,momentum=0.9,clipnorm=1)



model.compile(optimizer=sgd,loss='categorical_crossentropy',metrics=['accuracy'])

checkpoint = ModelCheckpoint("VGG_SGD_no_reg_weights_amit.hdf5", monitor='val_accuracy', verbose=1, save_best_only=True, save_weights_only=True, mode='auto', period=1)
early = EarlyStopping(monitor='val_accuracy', min_delta=0, patience=5, verbose=1, mode='auto',restore_best_weights=True)


In [3]:
model.fit(aug_data.flow(x_train, y_train, batch_size=128), batch_size=128, epochs=100, verbose=1, validation_data=(x_test, y_test),callbacks=[checkpoint,early])


Epoch 1/100
391/391 [==============================] - ETA: 0s - loss: 4.2166 - accuracy: 0.0595
Epoch 00001: val_accuracy improved from -inf to 0.11750, saving model to VGG_SGD_no_reg_weights_amit.hdf5
391/391 [==============================] - 310s 792ms/step - loss: 4.2166 - accuracy: 0.0595 - val_loss: 3.7561 - val_accuracy: 0.1175
Epoch 2/100
391/391 [==============================] - ETA: 0s - loss: 3.6302 - accuracy: 0.1511
Epoch 00002: val_accuracy improved from 0.11750 to 0.22570, saving model to VGG_SGD_no_reg_weights_amit.hdf5
391/391 [==============================] - 323s 826ms/step - loss: 3.6302 - accuracy: 0.1511 - val_loss: 3.2376 - val_accuracy: 0.2257
Epoch 3/100
391/391 [==============================] - ETA: 0s - loss: 3.2428 - accuracy: 0.2191
Epoch 00003: val_accuracy improved from 0.22570 to 0.27380, saving model to VGG_SGD_no_reg_weights_amit.hdf5
391/391 [==============================] - 323s 825ms/step - loss: 3.2428 - accuracy: 0.2191 - val_loss: 2.9444 - v

In [4]:
y_pred=model.predict_classes(x_test)
from sklearn.metrics import classification_report,accuracy_score,recall_score,precision_score
print(classification_report(Y_test,y_pred))
print("Accuracy is {}".format(accuracy_score(Y_test,y_pred)))
print("Precision is {}".format(precision_score(Y_test,y_pred,average="weighted")))
print("Recall is {}".format(recall_score(Y_test,y_pred,average="weighted")))

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
              precision    recall  f1-score   support

           0       0.94      0.72      0.81       100
           1       0.90      0.69      0.78       100
           2       0.51      0.55      0.53       100
           3       0.56      0.42      0.48       100
           4       0.51      0.34      0.41       100
           5       0.73      0.59      0.65       100
           6       0.70      0.57      0.63       100
           7       0.76      0.62      0.68       100
           8       0.68      0.84      0.75       100
           9       0.71      0.77      0.74       100
          10       0.52      0.29      0.37       100
          11     

In [5]:
YY=model.predict_classes(x_train)
print("Accuracy is {}".format(accuracy_score(Y_train,YY)))

Accuracy is 0.8882


In [6]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 64)        1792      
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 32, 32, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 64)        36928     
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 32, 32, 64)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 31, 31, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 31, 31, 128)       73856     
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 31, 31, 128)       0

In [1]:
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Flatten, Dense,BatchNormalization,Activation,Dropout,LeakyReLU
from tensorflow.keras.optimizers import SGD,Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical
from tensorflow.keras.datasets import cifar100



(x_train, Y_train), (x_test, Y_test) = cifar100.load_data()
#x_train = x_train.astype('float32') / 255
#x_test = x_test.astype('float32') / 255

y_train = to_categorical(Y_train)
y_test = to_categorical(Y_test)


#import tensorflow as 
model = Sequential()

# Creating first block- (2 Convolution + 1 Max pool)
model.add(Conv2D(filters= 64, kernel_size= (3,3), strides= (1,1), padding='same', input_shape= (32, 32, 3)))
model.add(LeakyReLU(alpha=0.1))
model.add(Conv2D(filters= 64, kernel_size= (3,3), strides= (1,1), padding='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPool2D(pool_size= (2,2), strides=(1,1)))

# Creating second block- (2 Convolution + 1 Max pool)
model.add(Conv2D(filters= 128, kernel_size= (3,3), strides= (1,1), padding='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(Conv2D(filters= 128, kernel_size= (3,3), strides= (1,1), padding='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPool2D(pool_size= (2,2), strides=(1,1)))


# Creating third block- (3 Convolution + 1 Max pool)
model.add(Conv2D(filters= 256, kernel_size= (3,3), strides= (1,1), padding='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(Conv2D(filters= 256, kernel_size= (3,3), strides= (1,1), padding='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(Conv2D(filters= 256, kernel_size= (3,3), strides= (1,1), padding='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPool2D(pool_size= (2,2), strides=(1,1)))

# Creating fourth block- (3 Convolution + 1 Max pool)
model.add(Conv2D(filters= 512, kernel_size= (3,3), strides= (1,1), padding='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(Conv2D(filters= 512, kernel_size= (3,3), strides= (1,1), padding='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(Conv2D(filters= 512, kernel_size= (3,3), strides= (1,1), padding='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPool2D(pool_size= (2,2), strides=(2,2)))

# Creating fifth block- (3 Convolution + 1 Max pool)
model.add(Conv2D(filters= 512, kernel_size= (3,3), strides= (1,1), padding='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(Conv2D(filters= 512, kernel_size= (3,3), strides= (1,1), padding='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(Conv2D(filters= 512, kernel_size= (3,3), strides= (1,1), padding='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPool2D(pool_size= (2,2), strides=(1,1)))

# Flattening the pooled image pixels
model.add(Flatten())

# Creating 2 Dense Layers
model.add(Dense(units= 512))
model.add(LeakyReLU(alpha=0.1))

model.add(Dense(units= 100, activation='softmax'))

model.load_weights("../weights/VGG_SGD_no_reg_weights_amit.hdf5")

y_true = y_test.argmax(-1)
y_pred = model.predict(x_test).argmax(-1)

from sklearn.metrics import classification_report,accuracy_score,recall_score,precision_score
print(classification_report(Y_test,y_pred))
print("Accuracy is {}".format(accuracy_score(Y_test,y_pred)))
print("Precision is {}".format(precision_score(Y_test,y_pred,average="weighted")))
print("Recall is {}".format(recall_score(Y_test,y_pred,average="weighted")))

Using TensorFlow backend.




              precision    recall  f1-score   support

           0       0.94      0.72      0.81       100
           1       0.90      0.69      0.78       100
           2       0.51      0.55      0.53       100
           3       0.56      0.42      0.48       100
           4       0.51      0.34      0.41       100
           5       0.73      0.59      0.65       100
           6       0.70      0.57      0.63       100
           7       0.76      0.62      0.68       100
           8       0.68      0.84      0.75       100
           9       0.71      0.77      0.74       100
          10       0.52      0.29      0.37       100
          11       0.35      0.54      0.42       100
          12       0.63      0.75      0.68       100
          13       0.59      0.47      0.53       100
          14       0.51      0.69      0.58       100
          15       0.58      0.60      0.59       100
          16       0.52      0.68      0.59       100
          17       0.78  